In [1]:
! pip install finance-datareader

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import itertools
import random

import FinanceDataReader as fdr
from tqdm import tqdm
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
random.seed(1234)
np.random.seed(1234)
os.environ["PYTHONHASHSEED"] = str(1234)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# GridSearch로 찾은 최적의 파라미터로 예측 시행

In [5]:
path = '/content/drive/MyDrive/주식 종가 예측/open/'
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))

start_date = '20210104'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

In [6]:
submission_name = 'sample_submission.csv'
submission = pd.read_csv(os.path.join(path,submission_name))

In [7]:
nmae_df = pd.DataFrame(columns = ['code', 'nmae'])
error_df = pd.DataFrame(columns = ['code'])

In [ ]:
import warnings
import itertools
import time
from statsmodels.tsa.statespace.sarimax import SARIMAX

# ignore warnings
warnings.filterwarnings("ignore")

for code in tqdm(stock_list['종목코드'].values):
  print(f'\n code: {code} \n')

  # Load data
  stock_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_data.fillna(method = 'ffill', inplace = True)

  # Split the data into train / validation / test sets
  train_public = stock_data[:-20] # ~ 10/29 : predict from 11/1 to 11/5
  train_private = stock_data # ~ 11/26 : predict from 11/29 to 12/3

  test_public = stock_data[-20:-15]['Close'].reset_index(drop = True) # from 11/1 to 11/5
  test_public = np.array(test_public)

  gridsearch_result = pd.read_csv(f'/content/drive/MyDrive/주식 종가 예측/sarimax_gridsearch_final/{code}_sarimax_gridsearch.csv')

  min_idx = gridsearch_result[['aic']].idxmin()

  pdq = gridsearch_result.loc[min_idx, 'pdq'].item()
  p = np.int64(pdq[1])
  d = np.int64(pdq[4])
  q = np.int64(pdq[7])
  optimal_pdq = (p, d, q)

  pdqs = gridsearch_result.loc[min_idx, 'seasonal_pdqs'].item()
  seasonal_p = np.int64(pdqs[1])
  seasonal_d = np.int64(pdqs[4])
  seasonal_q = np.int64(pdqs[7])
  s = np.int64(pdqs[10])
  optimal_pdqs = (seasonal_p, seasonal_d, seasonal_q, s)

  optimal_trend = gridsearch_result.loc[min_idx, 'trend_param'].item()

  try:
    model_public = SARIMAX(train_public['Close'].values.astype('float64'), exog = None, order = optimal_pdq, seasonal_order = optimal_pdqs,
                         trend = optimal_trend, enforce_stationarity = True, enforce_invertibility = True)
    model_public_fit = model_public.fit()

    forecast_public = model_public_fit.forecast(steps = 5) # predict from 11/1 to 11/5
    pred_public = forecast_public

    nmae = np.mean(np.abs(test_public - pred_public) / test_public)
    nmae_df = nmae_df.append({'code' : code, 'nmae' : nmae}, ignore_index = True)

    print("pdq : ", optimal_pdq, "\n")
    print("pdqs : ", optimal_pdqs, "\n")
    print("trend : ", optimal_trend, "\n")
    print("public prediction result : ", pred_public, "\n")
    print("public actual value : ", test_public, "\n")
    print("private prediction result : ", pred_private, "\n")
    print("nmae * 100 : ", nmae * 100, "\n")

    # Add public predicted value to the submission file
    for i in range(0, 5):
      submission.loc[i, code] = pred_public[i]
  
  except:
    error_df = error_df.append({'code' : code}, ignore_index = True)

  try:
    model_private = SARIMAX(train_private['Close'].values.astype('float64'), exog = None, order = optimal_pdq, seasonal_order = optimal_pdqs,
                            trend = optimal_trend, enforce_seasonality = True, enforce_invertivility = True)
    model_private_fit = model_private.fit()

    forecast_private = model_private_fit.forecast(steps = 5) # predict from 11/29 to 12/3
    pred_private = forecast_private

    for i in range(0, 5):
      submission.loc[i + 5, code] = pred_private[i]
  
  except:
    error_df = error_df.append({'code' : code}, ignore_index = True)

  time.sleep(1)  

  0%|          | 0/370 [00:00<?, ?it/s]


 code: 005930 

pdq :  (2, 1, 2) 

pdqs :  (2, 0, 2, 5) 

trend :  c 

public prediction result :  [69605.49934626 69477.42012356 69610.44802139 69646.43687876
 69522.82761396] 

public actual value :  [69900 71500 70400 70600 70200] 



  0%|          | 1/370 [00:09<56:51,  9.24s/it]


 code: 000660 

pdq :  (0, 1, 0) 

pdqs :  (2, 2, 1, 5) 

trend :  t 

public prediction result :  [103334.69685037 104178.03202693 104684.26794825 106751.68435606
 107473.5072473 ] 

public actual value :  [106500 107500 105500 106000 107000] 

private prediction result :  [72419.37469584 72309.32040781 72451.11755533 72410.70486183
 72126.00267288] 

nmae * 100 :  1.5974380915988018 



  1%|          | 2/370 [00:15<46:36,  7.60s/it]

...


 code: 012330 

pdq :  (1, 1, 0) 

pdqs :  (1, 2, 1, 5) 

trend :  t 

public prediction result :  [255149.09073744 256672.89857616 252837.35886122 250765.76422875
 243108.16612308] 

public actual value :  [253000 255500 252500 252000 252500] 

private prediction result :  [168574.65324526 169041.03782996 169539.92712566 168298.55032468
 167051.40155135] 

nmae * 100 :  1.1302849651756626 



  4%|▍         | 15/370 [01:21<29:47,  5.04s/it]


 code: 096770 



  4%|▍         | 16/370 [01:25<27:28,  4.66s/it]


 code: 051900 

